# Lab | Data Structuring and Combining Data

Objective: 
- Combine and integrate data from multiple sources using merging, concatenating, or joining techniques to generate more comprehensive and meaningful datasets for analysis.
- Modify the structure of data by pivoting, stacking/unstacking, or melting dataframes, enabling them to efficiently explore and analyze complex datasets. 

# Challenge 1: Combining Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the following links: 
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv


Perform data cleaning and formatting using the main cleaning and formatting function created in the previous lab to clean and format the data.

Combine the data from the three dataframes into a single dataframe, named "customer_data", using appropriate merging, concatenating, and joining techniques.

Verify that the customer_data dataframe contains all the rows and columns from the three original dataframes.

Observation: 
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [1]:
import pandas as pd

file1 = pd.read_csv("file1.csv")
file2 = pd.read_csv("file2.csv")
file3 = pd.read_csv("file3.csv")

def clean_and_format_data(df):
    '''
    Perform data cleaning and formatting operations on df
    '''
    cleaned_df = df.dropna().drop_duplicates()
    return cleaned_df

file1_cleaned = clean_and_format_data(file1)
file2_cleaned = clean_and_format_data(file2)
file3_cleaned = clean_and_format_data(file3)

customer_data = pd.concat([file1_cleaned, file2_cleaned, file3_cleaned], ignore_index=True)

print(customer_data.shape)
print(customer_data.head())




(9010, 13)
  Customer          ST GENDER             Education Customer Lifetime Value  \
0  QZ44356     Arizona      F              Bachelor              697953.59%   
1  AI49188      Nevada      F              Bachelor             1288743.17%   
2  WW63253  California      M              Bachelor              764586.18%   
3  GA49547  Washington      M  High School or Below              536307.65%   
4  OC83172      Oregon      F              Bachelor              825629.78%   

    Income  Monthly Premium Auto Number of Open Complaints     Policy Type  \
0      0.0                  94.0                    1/0/00   Personal Auto   
1  48767.0                 108.0                    1/0/00   Personal Auto   
2      0.0                 106.0                    1/0/00  Corporate Auto   
3  36357.0                  68.0                    1/0/00   Personal Auto   
4  62902.0                  69.0                    1/0/00   Personal Auto   

   Vehicle Class  Total Claim Amount State Ge

# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

## Exercise 1: Clean and Format the Data

While the dataset has been partially cleaned and formatted, we still need to perform several data cleaning tasks. Specifically, we need to standardize the column names, clean null values, convert the effective_to_date column to datetime, and extract the months from the dataset and store them in a separate column. 

To accomplish these tasks, we will use the functions created in the previous step to standardize the column names and deal with null values, and then we will apply additional functions to convert the effective_to_date column to datetime and extract the months.

Save the clean dataset into a csv file.

In [2]:
import pandas as pd
import numpy as np

def standardize_column_names(df):
    df.columns = df.columns.str.replace(' ', '_').str.lower()
    return df

data = pd.read_csv("marketing_customer_analysis.csv")
data = standardize_column_names(data)
data = data.dropna()
data['effective_to_date'] = pd.to_datetime(data['effective_to_date'])
data['month'] = data['effective_to_date'].dt.month

data.to_csv("cleaned_marketing_data.csv", index=False)

## Exercise 2: Structuring Data

1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

3. You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table. 

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [15]:
sales_channel_revenue = data.pivot_table(values='total_claim_amount', index='sales_channel', aggfunc='sum').round(2)

print(sales_channel_revenue)

               total_claim_amount
sales_channel                    
Agent                   757063.79
Branch                  547893.57
Call Center             391964.55
Web                     285995.48


In [16]:
clv_table = data.pivot_table(values='customer_lifetime_value', index=['gender', 'education'], aggfunc='mean').round(2)

print(clv_table)


                             customer_lifetime_value
gender education                                    
F      Bachelor                              8039.29
       College                               7798.08
       Doctor                                8475.30
       High School or Below                  8475.52
       Master                                8243.13
M      Bachelor                              8125.26
       College                               7984.30
       Doctor                                7659.74
       High School or Below                  7730.15
       Master                                8013.87


In [18]:
complaints_table = data.pivot_table(values='number_of_open_complaints', index='policy_type', columns='month', aggfunc='sum').fillna(0)

complaints_table_long = complaints_table.reset_index().melt(id_vars='policy_type', var_name='month', value_name='complaints')

print(complaints_table_long)


      policy_type month  complaints
0  Corporate Auto     1       161.0
1   Personal Auto     1       725.0
2    Special Auto     1        33.0
3  Corporate Auto     2       176.0
4   Personal Auto     2       623.0
5    Special Auto     2        37.0
